Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
import cv2
import pickle

The dataset with 25000 images was loaded on to Google Colab, the data retrieved and stored as pickle file.
It is now retrieved on this jupyter notebook

In [6]:
with open("data.pkl", "rb") as f:
    loaded_data = pickle.load(f)

In [8]:
loaded_data = np.array(loaded_data, dtype=object)

In [18]:
id = np.array([row[0] for row in loaded_data],dtype= np.uint32)
pixels = np.array([row[1] for row in loaded_data],dtype= np.uint8)
label = np.array([row[2] for row in loaded_data],dtype= np.uint8)

In [24]:
datatype = [("id",np.uint32),("pixels",np.uint8,(128,128,3)),("label",np.uint8)]

In [26]:
dataset = np.array(list(zip(id,pixels,label)),dtype=datatype)

In [28]:
np.save('dataset.npy',dataset)

In [30]:
len(dataset)

24946

In [32]:
dataset = np.load("dataset.npy", allow_pickle=True)

In [34]:
dataset

array([( 3707, [[[ 96, 125, 134], [ 96, 125, 134], [ 99, 124, 134], ..., [171, 179, 171], [172, 179, 172], [170, 177, 170]], [[ 96, 125, 134], [ 96, 124, 134], [100, 125, 135], ..., [173, 180, 173], [173, 180, 173], [172, 179, 172]], [[ 94, 123, 132], [ 96, 124, 134], [ 98, 123, 133], ..., [181, 187, 181], [174, 181, 174], [170, 178, 170]], ..., [[162, 190, 197], [161, 190, 198], [152, 182, 193], ..., [128, 143, 149], [139, 152, 155], [132, 144, 148]], [[176, 200, 205], [176, 200, 207], [158, 185, 192], ..., [131, 146, 151], [134, 146, 150], [124, 135, 136]], [[176, 195, 202], [173, 192, 199], [178, 197, 204], ..., [137, 149, 152], [135, 144, 147], [ 97,  97,  97]]], 0),
       ( 8927, [[[165, 158, 109], [160, 156, 106], [165, 160, 111], ..., [158, 156, 102], [160, 159, 103], [168, 167, 111]], [[163, 156, 106], [164, 160, 109], [167, 162, 112], ..., [162, 160, 106], [162, 161, 105], [159, 158, 102]], [[160, 156, 100], [164, 161, 106], [168, 164, 112], ..., [161, 159, 105], [158, 157, 1

In [36]:
len(dataset)

24946

In [38]:
np.random.shuffle(dataset)

The dataset was randomly shuffled as the earlier data set contained the images of dogs and cats seperately.

In [40]:
dataset

array([(11419, [[[ 29,  37,  42], [ 39,  48,  52], [ 32,  43,  47], ..., [ 56,  70,  76], [ 34,  51,  57], [ 47,  67,  72]], [[ 41,  50,  54], [ 34,  44,  48], [ 37,  48,  52], ..., [ 68,  82,  89], [ 53,  71,  76], [ 30,  50,  55]], [[ 43,  51,  58], [ 46,  56,  63], [ 48,  60,  66], ..., [ 59,  75,  82], [ 50,  68,  75], [ 46,  65,  72]], ..., [[ 44,  57,  65], [ 38,  55,  59], [ 38,  47,  48], ..., [110, 128, 139], [108, 129, 141], [ 60,  86, 100]], [[ 43,  56,  64], [ 58,  75,  79], [ 48,  59,  60], ..., [100, 118, 129], [ 83, 105, 116], [ 80, 105, 121]], [[ 37,  50,  58], [ 37,  54,  58], [ 52,  63,  64], ..., [ 89, 107, 118], [ 79, 101, 113], [103, 129, 144]]], 0),
       (10683, [[[  3,  10,   7], [  3,  10,   7], [  8,  15,  12], ..., [  4,  18,  14], [  4,  18,  14], [  4,  19,  11]], [[  1,   8,   5], [  1,   8,   5], [  2,   9,   6], ..., [  4,  18,  14], [  4,  18,  14], [  5,  20,  13]], [[  2,   9,   6], [  2,   9,   6], [  4,  11,   8], ..., [  4,  18,  14], [  4,  18,  

The pixels column is taken as input X and the labels is the output Y.

The pixels column contains of a 128*128*3 pixels array which contains the intensity og RGB colors.

As these values of intensity range from 0 to 255 they are divided by 255 to normalise the input to get values between 0 and 1.

In [15]:
X = np.stack(dataset['pixels'])
Y = np.stack(dataset['label'])

In [16]:
X = X/255.0

In [17]:
X_train = X.astype("float16")

Creating the model

This includes 5 layers: 3 convulational layers and 2 dense layers.

The convulational layers help to identify patterns or parts in the image that relate to cats or dogs.

The output layer is sigmoid to predict the binary output of the neural network.

In [42]:
predictor_model = models.Sequential([

    layers.Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1,activation='sigmoid')
])    

C:\Users\Sanat\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
predictor_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']
             )

In [46]:
predictor_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
#predictor_model.fit(X_train,Y,epochs=10,batch_size=1000,validation_split=0.2)

MemoryError: Unable to allocate 3.65 GiB for an array with shape (19956, 49152) and data type float32

A batch generator function is defined which takes the complete data and provides it to the model in batches so that the training process and computation can be simplified.

In [66]:
def data_generator(data,batch_size):
    while True:
        for i in range(0,len(data),batch_size):
            batch = data[i:i+batch_size]
            pixel = np.array([item[1] for item in batch]) / 255.0
            label = np.array([item[2] for item in batch])
            yield pixel, label
        

In [50]:
dataset

array([(11419, [[[ 29,  37,  42], [ 39,  48,  52], [ 32,  43,  47], ..., [ 56,  70,  76], [ 34,  51,  57], [ 47,  67,  72]], [[ 41,  50,  54], [ 34,  44,  48], [ 37,  48,  52], ..., [ 68,  82,  89], [ 53,  71,  76], [ 30,  50,  55]], [[ 43,  51,  58], [ 46,  56,  63], [ 48,  60,  66], ..., [ 59,  75,  82], [ 50,  68,  75], [ 46,  65,  72]], ..., [[ 44,  57,  65], [ 38,  55,  59], [ 38,  47,  48], ..., [110, 128, 139], [108, 129, 141], [ 60,  86, 100]], [[ 43,  56,  64], [ 58,  75,  79], [ 48,  59,  60], ..., [100, 118, 129], [ 83, 105, 116], [ 80, 105, 121]], [[ 37,  50,  58], [ 37,  54,  58], [ 52,  63,  64], ..., [ 89, 107, 118], [ 79, 101, 113], [103, 129, 144]]], 0),
       (10683, [[[  3,  10,   7], [  3,  10,   7], [  8,  15,  12], ..., [  4,  18,  14], [  4,  18,  14], [  4,  19,  11]], [[  1,   8,   5], [  1,   8,   5], [  2,   9,   6], ..., [  4,  18,  14], [  4,  18,  14], [  5,  20,  13]], [[  2,   9,   6], [  2,   9,   6], [  4,  11,   8], ..., [  4,  18,  14], [  4,  18,  

In [52]:
split_index = int(0.8*len(dataset))

In [54]:
split_index

19956

The dataset is divided into training and validation set

In [56]:
train_data = dataset[0:split_index]
validation_data = dataset[split_index:]

In [58]:
len(train_data)

19956

In [68]:
train_gen = data_generator(train_data,batch_size=32)
val_gen = data_generator(validation_data,batch_size=32)

The model is fit on the training data with validation on the validation set

In [70]:
predictor_model.fit(train_gen,
          steps_per_epoch=len(train_data)//32,
          validation_data=val_gen,
          validation_steps=len(validation_data)//32,
          epochs=10)

Epoch 1/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 92s 148ms/step - accuracy: 0.7287 - loss: 0.5358 - val_accuracy: 0.7911 - val_loss: 0.4446
Epoch 2/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 95s 152ms/step - accuracy: 0.7840 - loss: 0.4636 - val_accuracy: 0.8054 - val_loss: 0.4178
Epoch 3/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 93s 150ms/step - accuracy: 0.8276 - loss: 0.3835 - val_accuracy: 0.8148 - val_loss: 0.4040
Epoch 4/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 92s 147ms/step - accuracy: 0.8574 - loss: 0.3273 - val_accuracy: 0.8084 - val_loss: 0.4442
Epoch 5/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 94s 151ms/step - accuracy: 0.8843 - loss: 0.2694 - val_accuracy: 0.8201 - val_loss: 0.4637
Epoch 6/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 98s 157ms/step - accuracy: 0.9063 - loss: 0.2278 - val_accuracy: 0.8415 - val_loss: 0.4488
Epoch 7/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 91s 147ms/step - accuracy: 0.9234 - loss: 0.1896 - val_accuracy: 0.8322 - val_loss: 0.4772
Epoch 8/10
623/623 ━━━━━━━━━━━━━━━━━━━━ 91s 146ms/step - accuracy: 0.9376 - loss: 0

In [74]:
predictor_model.save("predictor_model.keras")

In [76]:
data_path = r"C:\Users\Sanat\OneDrive\Desktop\PROGRAMMING\projectWork\catVSdog\test1"

The following code is to carry out the prediction on the test dataset. Unfortunately the correct output predictions of the testing dataset were not provided in the kaggle directory so it was not possible to measure the accuracy of the model on the testing data.

In [86]:
img_size = 128
data = []
for img in os.listdir(data_path):
    img_path = os.path.join(data_path,img)
    id = int(img.replace(".jpg",""))  
    img_arr = cv2.imread(img_path)
    img_arr = cv2.resize(img_arr,(img_size,img_size))
    data.append([id,img_arr])

In [90]:
len(data)

12500

In [98]:
test_ids = np.array([item[0] for item in data], dtype=np.uint32)
test_images = np.array([item[1] for item in data], dtype=np.uint8)

In [100]:
test_images = test_images/255.0

In [102]:
y_predicted = predictor_model.predict(test_images)

391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 51ms/step


In [104]:
len(y_predicted)

12500

In [110]:
predicted_classes = (y_predicted > 0.5).astype("int")

In [114]:
predicted_classes

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]])

In [116]:
data_type = [("id",np.uint32),("label",np.uint8)]
test_results = np.array(list(zip(test_ids,predicted_classes)),dtype=data_type)

C:\Users\Sanat\AppData\Local\Temp\ipykernel_16736\1263620585.py:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_results = np.array(list(zip(test_ids,predicted_classes)),dtype=data_type)


In [118]:
test_results

array([(   1, 1), (  10, 0), ( 100, 0), ..., (9997, 1), (9998, 0),
       (9999, 0)], dtype=[('id', '<u4'), ('label', 'u1')])

In [122]:
len(test_results)

12500

In [128]:
test_results.dtype()

TypeError: 'numpy.dtypes.VoidDType' object is not callable

In [130]:
test_results_final = pd.DataFrame(test_results)

In [132]:
test_results_final

,id,label
0,1,1
1,10,0
2,100,0
3,1000,1
4,10000,1
...,...,...
12495,9995,0
12496,9996,1
12497,9997,1
12498,9998,0


In [136]:
df_sorted = test_results_final.sort_values(by="id").reset_index(drop=True)

In [138]:
df_sorted

,id,label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
12495,12496,0
12496,12497,0
12497,12498,1
12498,12499,1


In [140]:
df_sorted.to_csv('final_results')

END OF CODE